In [1]:
import numpy as np
import netCDF4
import h5netcdf
import xarray as xr
import sys
import tensorflow as tf
from tensorflow import keras
import pickle
from os.path import join
sys.path.append('/home/samuel.varga/projects/deep_learning/')

from deep_learning.training_utils import load_rotation, convert_to_tf, resize_neural_net
from deep_learning.deep_networks import create_U_net_classifier_2D

2023-12-24 16:48:13.245745: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-24 16:48:13.589387: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
from keras import backend as K
import gc
from numba import cuda
outdir='/work/samuel.varga/projects/2to6_hr_severe_wx/DEEP_LEARNING/'

In [25]:
batch_size=1024
target_column='tornado_severe__36km'
rotation=0
X_train, y_train, mean, variance = load_rotation(join('/work/samuel.varga/data/2to6_hr_severe_wx/DEEP_LEARNING/',f'wofs_dl_severe__2to6hr__rot_{rotation}__training_data'), rotation, target_column)
X_val, y_val = load_rotation(join('/work/samuel.varga/data/2to6_hr_severe_wx/DEEP_LEARNING/',f'wofs_dl_severe__2to6hr__rot_{rotation}__validation_data'), rotation, target_column)
X_test, y_test = load_rotation(join('/work/samuel.varga/data/2to6_hr_severe_wx/DEEP_LEARNING/',f'wofs_dl_severe__2to6hr__testing_data'), None, target_column)
print(np.shape(X_train))
print(np.shape(y_train))

#Convert to tf dataset
train_ds = convert_to_tf((X_train,np.expand_dims(y_train, axis=-1)), batch_size)
val_ds = convert_to_tf((X_val, np.expand_dims(y_val, axis=-1)), batch_size)
test_ds=convert_to_tf((X_test, np.expand_dims(y_test, axis=-1)))

Training path detected - loading scaling
(6350, 16, 16, 63)
(6350, 16, 16)


In [26]:
#U-net architectural parameters
conv_filters=[64,128,256,512]
conv_size=[2,1,2,1]
max_pool=[2,2,2,2]
conv_layers =[{'filters': f, 'kernel_size': (s), 'pool_size': (p), 'strides': (p)} if p > 1
                   else {'filters': f, 'kernel_size': (s), 'pool_size': None, 'strides': None}
                   for s, f, p, in zip(conv_size, conv_filters, max_pool)]
args={'lrate':1e-4, 'loss':'binary_crossentropy','activation_conv':'relu','activation_out':'sigmoid',
     'p_spatial_dropout':0.1, 'filters':conv_filters, 'size':conv_size, 'pool':max_pool, 'shape':(16,16),
     'rotation':rotation,'target_column':target_column}
thresholds= [0.15]
metrics=[tf.keras.metrics.SparseCategoricalAccuracy(), tf.keras.metrics.MeanSquaredError(name='Brier score'),
    tf.keras.metrics.AUC(name='auc'), tf.keras.metrics.AUC(name='prc', curve='PR'),     
         tf.keras.metrics.FalseNegatives(thresholds=thresholds), tf.keras.metrics.FalsePositives(thresholds=thresholds),
         tf.keras.metrics.MeanAbsoluteError(), tf.keras.metrics.TrueNegatives(thresholds=thresholds),
         tf.keras.metrics.TruePositives(thresholds=thresholds)]

In [150]:
#Create U-net
u_net = create_U_net_classifier_2D(image_size=args['shape'], nchannels=63, n_classes=1, conv_layers=conv_layers, p_spatial_dropout=args['p_spatial_dropout'], metrics=metrics,
                               lrate=args['lrate'], loss=args['loss'], activation_conv=args['activation_conv'], activation_out='sigmoid',
                                      normalization=(mean, variance))

Model: "model_9"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 Input (InputLayer)             [(None, 16, 16, 63)  0           []                               
                                ]                                                                 
                                                                                                  
 normalization_9 (Normalization  (None, 16, 16, 63)  0           ['Input[0][0]']                  
 )                                                                                                
                                                                                                  
 Encode_0_0 (Conv2D)            (None, 16, 16, 64)   16192       ['normalization_9[0][0]']        
                                                                                            

 De_Sp_Dr_2_1 (SpatialDropout2D  (None, 4, 4, 256)   0           ['Decode_2_1[0][0]']             
 )                                                                                                
                                                                                                  
 Decode_2_0 (Conv2D)            (None, 4, 4, 256)    262400      ['De_Sp_Dr_2_1[0][0]']           
                                                                                                  
 De_Sp_Dr_2_0 (SpatialDropout2D  (None, 4, 4, 256)   0           ['Decode_2_0[0][0]']             
 )                                                                                                
                                                                                                  
 Decode_Upsample_1 (UpSampling2  (None, 8, 8, 256)   0           ['De_Sp_Dr_2_0[0][0]']           
 D)                                                                                               
          

In [151]:
#Callbacks
early_stopping_cb =keras.callbacks.EarlyStopping(patience=25, restore_best_weights=True,
                                                min_delta=0.001, monitor='val_loss')
tensorboard_cb= keras.callbacks.TensorBoard(log_dir=join(outdir,'logs'), histogram_freq=1)

In [152]:
#Learn the model
history = u_net.fit(train_ds, epochs=1, verbose=True, validation_data = val_ds,
    callbacks=[early_stopping_cb, tensorboard_cb])

2023-12-24 18:27:20.486267: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel_9/En_Sp_Dr_0_0/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


7/7 [==============================] - 2s 134ms/step - loss: 0.6097 - sparse_categorical_accuracy: 0.9940 - Brier score: 0.2031 - auc: 0.3939 - prc: 0.0070 - false_negatives_3: 381.0000 - false_positives_3: 2003426.0000 - mean_absolute_error: 0.4475 - true_negatives_3: 1821.0000 - true_positives_3: 11652.0000 - val_loss: 0.5227 - val_sparse_categorical_accuracy: 0.9941 - val_Brier score: 0.1668 - val_auc: 0.3685 - val_prc: 0.0047 - val_false_negatives_3: 0.0000e+00 - val_false_positives_3: 389327.0000 - val_mean_absolute_error: 0.4041 - val_true_negatives_3: 60.0000 - val_true_positives_3: 2293.0000


In [31]:
fbase = f"{args['target_column']}_Rot_{args['rotation']}_{args['shape'][0]}_{args['shape'][1]}_lrate_{args['lrate']}_spatial_dropout_{args['p_spatial_dropout']}_filters_{args['filters']}_size_{args['size']}_pool_{args['pool']}"
results = {}
results['args'] = args
results['predict_val'] = u_net.predict(val_ds)
results['predict_val_eval'] = u_net.evaluate(val_ds)

if test_ds is not None:
    results['predict_test']=u_net.predict(test_ds)
    results['predict_test_eval']=u_net.evaluate(test_ds)

results['predict_train']=u_net.predict(train_ds)
results['predict_train_eval']=u_net.evaluate(train_ds)
results['history']=history.history
results['fname_base']=fbase

#Save results
with open(join(join(outdir, 'results'), f'{fbase}_results.pkl'),'wb') as fp:
    pickle.dump(results, fp)

#save model
if False:
    u_net.save(join(join(outdir, 'models'), f'{fbase}_model'))
print(np.max(results['predict_val']))
print(np.mean(results['predict_val']))
print(fbase)

7/7 [==============================] - 0s 20ms/step - loss: 0.0249 - sparse_categorical_accuracy: 0.9940 - Brier score: 0.0057 - auc: 0.9336 - prc: 0.1759 - false_negatives_3: 5766.0000 - false_positives_3: 23724.0000 - mean_absolute_error: 0.0138 - true_negatives_3: 1592136.0000 - true_positives_3: 3974.0000
0.5302415
0.008179982
tornado_severe__36km_Rot_0_16_16_lrate_0.0001_spatial_dropout_0.1_filters_[64, 128, 256, 512]_size_[2, 1, 2, 1]_pool_[2, 2, 2, 2]


In [ ]:
#Hyperparam search
from itertools import product
for rotation, p_s, lrate, cs in product([0, 1, 2 ,3, 4], [0.001, 0.01, 0.01, 0.1], [0.001, 0.01, 0.1], ([2,1,2,1],[2,2,2,2])):
    X_train, y_train, mean, variance = load_rotation(join('/work/samuel.varga/data/2to6_hr_severe_wx/DEEP_LEARNING/',f'wofs_dl_severe__2to6hr__rot_{rotation}__training_data'), rotation, target_column)
    X_val, y_val = load_rotation(join('/work/samuel.varga/data/2to6_hr_severe_wx/DEEP_LEARNING/',f'wofs_dl_severe__2to6hr__rot_{rotation}__validation_data'), rotation, target_column)
    print(np.shape(X_train))
    print(np.shape(y_train))
    print(f'Validation Base Rate:{np.mean(y_val)}')
    print(f'Validation non-event Rate:{1-np.mean(y_val)}')
    #Convert to tf dataset
    train_ds = convert_to_tf((X_train,np.expand_dims(y_train, axis=-1)), batch_size)
    val_ds = convert_to_tf((X_val,np.expand_dims(y_val, axis=-1)), batch_size)
    
    #U-net architectural parameters
    conv_filters=[32,64,128,256]
    max_pool=[2,2,2,2]
    conv_layers =[{'filters': f, 'kernel_size': (s), 'pool_size': (p), 'strides': (p)} if p > 1
                       else {'filters': f, 'kernel_size': (s), 'pool_size': None, 'strides': None}
                       for s, f, p, in zip(cs, conv_filters, max_pool)]
    args={'lrate':lrate, 'loss':'binary_crossentropy','activation_conv':'relu','activation_out':'sigmoid',
         'p_spatial_dropout':p_s, 'filters':conv_filters, 'size':cs, 'pool':max_pool, 'shape':(16,16),
         'rotation':rotation,'target_column':target_column}
##
    #create u-net
    #Pick better loss/activation functions
    u_net = create_U_net_classifier_2D(image_size=args['shape'], nchannels=63, n_classes=1, conv_layers=conv_layers, p_spatial_dropout=args['p_spatial_dropout'], metrics=metrics,
                               lrate=args['lrate'], loss=args['loss'], activation_conv=args['activation_conv'], activation_out=args['activation_out'],
                                      normalization=(mean, variance))
##
    early_stopping_cb =keras.callbacks.EarlyStopping(patience=30, restore_best_weights=True,
                                                    min_delta=0.001, monitor='val_loss')
    tensorboard_cb= keras.callbacks.TensorBoard(log_dir=join(outdir,f'logs/Rot_{rotation}_p_s_{p_s}_lrate_{lrate}_cs_{cs}'), histogram_freq=1)
    history = u_net.fit(train_ds, epochs=100, verbose=True, validation_data = val_ds,
        callbacks=[early_stopping_cb, tensorboard_cb])

Training path detected - loading scaling
(6350, 16, 16, 63)
(6350, 16, 16)
Validation Base Rate:0.047240093954248365
Validation non-event Rate:0.9527599060457517
Model: "model_114"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 Input (InputLayer)             [(None, 16, 16, 63)  0           []                               
                                ]                                                                 
                                                                                                  
 normalization_114 (Normalizati  (None, 16, 16, 63)  0           ['Input[0][0]']                  
 on)                                                                                              
                                                                                                  
 Encode_0_0 (Conv2D)       

 Decode_2_1 (Conv2D)            (None, 4, 4, 128)    196736      ['Concat2[0][0]']                
                                                                                                  
 De_Sp_Dr_2_1 (SpatialDropout2D  (None, 4, 4, 128)   0           ['Decode_2_1[0][0]']             
 )                                                                                                
                                                                                                  
 Decode_2_0 (Conv2D)            (None, 4, 4, 128)    65664       ['De_Sp_Dr_2_1[0][0]']           
                                                                                                  
 De_Sp_Dr_2_0 (SpatialDropout2D  (None, 4, 4, 128)   0           ['Decode_2_0[0][0]']             
 )                                                                                                
                                                                                                  
 Decode_Up

2023-12-20 18:36:35.758538: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel_114/En_Sp_Dr_0_0/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


7/7 [==============================] - 2s 156ms/step - loss: 0.6442 - sparse_categorical_accuracy: 0.9527 - Brier score: 0.1267 - auc: 0.6593 - prc: 0.0808 - false_negatives_4: 30843.0000 - false_positives_4: 1526562.0000 - mean_absolute_error: 0.2548 - true_negatives_4: 1551444.0000 - true_positives_4: 121871.0000 - val_loss: 0.3474 - val_sparse_categorical_accuracy: 0.9528 - val_Brier score: 0.0762 - val_auc: 0.4122 - val_prc: 0.0390 - val_false_negatives_4: 14332.0000 - val_false_positives_4: 117350.0000 - val_mean_absolute_error: 0.1639 - val_true_negatives_4: 255827.0000 - val_true_positives_4: 4171.0000
Epoch 2/100
7/7 [==============================] - 0s 51ms/step - loss: 0.2896 - sparse_categorical_accuracy: 0.9535 - Brier score: 0.0745 - auc: 0.5253 - prc: 0.0483 - false_negatives_4: 42659.0000 - false_positives_4: 597699.0000 - mean_absolute_error: 0.1836 - true_negatives_4: 952331.0000 - true_positives_4: 32911.0000 - val_loss: 0.2023 - val_sparse_categorical_accuracy: 0.95

7/7 [==============================] - 0s 51ms/step - loss: 0.1016 - sparse_categorical_accuracy: 0.9535 - Brier score: 0.0293 - auc: 0.9443 - prc: 0.5261 - false_negatives_4: 18150.0000 - false_positives_4: 95513.0000 - mean_absolute_error: 0.0588 - true_negatives_4: 1454517.0000 - true_positives_4: 57420.0000 - val_loss: 0.1421 - val_sparse_categorical_accuracy: 0.9528 - val_Brier score: 0.0395 - val_auc: 0.8828 - val_prc: 0.3226 - val_false_negatives_4: 7030.0000 - val_false_positives_4: 38340.0000 - val_mean_absolute_error: 0.0794 - val_true_negatives_4: 334837.0000 - val_true_positives_4: 11473.0000
Epoch 28/100
7/7 [==============================] - 0s 51ms/step - loss: 0.0987 - sparse_categorical_accuracy: 0.9535 - Brier score: 0.0285 - auc: 0.9485 - prc: 0.5441 - false_negatives_4: 16785.0000 - false_positives_4: 93602.0000 - mean_absolute_error: 0.0581 - true_negatives_4: 1456428.0000 - true_positives_4: 58785.0000 - val_loss: 0.1526 - val_sparse_categorical_accuracy: 0.9528 -

...

7/7 [==============================] - 0s 59ms/step - loss: 0.1011 - sparse_categorical_accuracy: 0.9549 - Brier score: 0.0296 - auc: 0.9455 - prc: 0.4884 - false_negatives_4: 18120.0000 - false_positives_4: 95736.0000 - mean_absolute_error: 0.0589 - true_negatives_4: 1441889.0000 - true_positives_4: 54495.0000 - val_loss: 0.1522 - val_sparse_categorical_accuracy: 0.9473 - val_Brier score: 0.0397 - val_auc: 0.8761 - val_prc: 0.3825 - val_false_negatives_4: 8700.0000 - val_false_positives_4: 30186.0000 - val_mean_absolute_error: 0.0731 - val_true_negatives_4: 355396.0000 - val_true_positives_4: 12758.0000
Epoch 28/100
7/7 [==============================] - 0s 59ms/step - loss: 0.1019 - sparse_categorical_accuracy: 0.9549 - Brier score: 0.0300 - auc: 0.9459 - prc: 0.4760 - false_negatives_4: 15945.0000 - false_positives_4: 112110.0000 - mean_absolute_error: 0.0621 - true_negatives_4: 1425515.0000 - true_positives_4: 56670.0000 - val_loss: 0.2012 - val_sparse_categorical_accuracy: 0.9473 

7/7 [==============================] - 0s 59ms/step - loss: 0.0887 - sparse_categorical_accuracy: 0.9549 - Brier score: 0.0265 - auc: 0.9591 - prc: 0.5640 - false_negatives_4: 13972.0000 - false_positives_4: 81360.0000 - mean_absolute_error: 0.0509 - true_negatives_4: 1456265.0000 - true_positives_4: 58643.0000 - val_loss: 0.1647 - val_sparse_categorical_accuracy: 0.9473 - val_Brier score: 0.0470 - val_auc: 0.8839 - val_prc: 0.3675 - val_false_negatives_4: 5198.0000 - val_false_positives_4: 56306.0000 - val_mean_absolute_error: 0.0968 - val_true_negatives_4: 329276.0000 - val_true_positives_4: 16260.0000
Epoch 54/100
7/7 [==============================] - 0s 59ms/step - loss: 0.0915 - sparse_categorical_accuracy: 0.9549 - Brier score: 0.0273 - auc: 0.9587 - prc: 0.5521 - false_negatives_4: 11574.0000 - false_positives_4: 108341.0000 - mean_absolute_error: 0.0574 - true_negatives_4: 1429284.0000 - true_positives_4: 61041.0000 - val_loss: 0.1902 - val_sparse_categorical_accuracy: 0.9473 

                                                                                                  
 Decode_3_0 (Conv2D)            (None, 2, 2, 256)    65792       ['De_Sp_Dr_3_1[0][0]']           
                                                                                                  
 De_Sp_Dr_3_0 (SpatialDropout2D  (None, 2, 2, 256)   0           ['Decode_3_0[0][0]']             
 )                                                                                                
                                                                                                  
 Decode_Upsample_2 (UpSampling2  (None, 4, 4, 256)   0           ['De_Sp_Dr_3_0[0][0]']           
 D)                                                                                               
                                                                                                  
 Concat2 (Concatenate)          (None, 4, 4, 384)    0           ['Decode_Upsample_2[0][0]',      
          

2023-12-20 19:03:07.300574: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel_188/En_Sp_Dr_0_0/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


7/7 [==============================] - 2s 96ms/step - loss: 1.3629 - sparse_categorical_accuracy: 0.9534 - Brier score: 0.0828 - auc: 0.5813 - prc: 0.0876 - false_negatives_4: 53791.0000 - false_positives_4: 618098.0000 - mean_absolute_error: 0.1706 - true_negatives_4: 1305109.0000 - true_positives_4: 40282.0000 - val_loss: 0.2568 - val_sparse_categorical_accuracy: 0.9473 - val_Brier score: 0.0527 - val_auc: 0.6962 - val_prc: 0.0954 - val_false_negatives_4: 16654.0000 - val_false_positives_4: 40216.0000 - val_mean_absolute_error: 0.0858 - val_true_negatives_4: 345366.0000 - val_true_positives_4: 4804.0000
Epoch 2/100
7/7 [==============================] - 0s 52ms/step - loss: 0.1917 - sparse_categorical_accuracy: 0.9549 - Brier score: 0.0496 - auc: 0.8171 - prc: 0.1795 - false_negatives_4: 20871.0000 - false_positives_4: 329070.0000 - mean_absolute_error: 0.1102 - true_negatives_4: 1208555.0000 - true_positives_4: 51744.0000 - val_loss: 0.1899 - val_sparse_categorical_accuracy: 0.9473 

In [75]:
K.clear_session()
gc.collect()
#gpu = cuda.get_current_device()
#gpu.reset()
cuda.close()